In [4]:
from azureml import Workspace

ws = Workspace()
ds = ws.datasets['Cleaned_ml_3000.csv']
frame = ds.to_dataframe()

In [5]:
frame

,Text,G/L
0,Investment Increase,45604
1,Investment Increase,45604
2,INV,113100
3,CHECK,113101
4,Hilfs Loehne,421000
5,Salaries Wages,430000
6,Mehrarbeit Gehlter,431000
7,Kraftfahrzeug Steuer,464000
8,Basic rent own use,470000
9,Own use OC flat rate,470000


In [6]:
frame.head(20)

,Text,G/L
0,Investment Increase,45604
1,Investment Increase,45604
2,INV,113100
3,CHECK,113101
4,Hilfs Loehne,421000
5,Salaries Wages,430000
6,Mehrarbeit Gehlter,431000
7,Kraftfahrzeug Steuer,464000
8,Basic rent own use,470000
9,Own use OC flat rate,470000


# Split of data

## Assumptions
    Short Data
    Text Bigram
    Use of IDF applied for tranformation

#### Taken few repetitive members of the dataset in train and rest in test

In [7]:
test_list = [40, 41, 42, 43, 44, 8, 9, 10, 11, 34, 35, 0]

In [9]:
train_list = [i not in test_list for i in list(range(frame.shape[0]))]

In [10]:
test = frame.iloc[test_list]

In [11]:
train = frame.iloc[train_list]

In [13]:
## Head View of training data
train.head()

,Text,G/L
1,Investment Increase,45604
2,INV,113100
3,CHECK,113101
4,Hilfs Loehne,421000
5,Salaries Wages,430000


In [14]:
## Head View of testing data
test.head()

,Text,G/L
40,Basic rent transfer,888900
41,OC flat rate trsfr own use,888900
42,Transfer basic rent own use,888900
43,Basic rent transfer,888900
44,OC flat rate trsfr own use,888900


### Data Transformation
    CountVectoriser  and
    TFIDF Transformer
    Label encoder

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [19]:
## Initializing TFIDF
tfidf = TfidfVectorizer(ngram_range=(1,3), use_idf=True)

In [20]:
## X_train and X_test formation
X_train = tfidf.fit_transform(train.Text)
X_test = tfidf.transform(test.Text)

In [22]:
## y_train and y_test formation
le = LabelEncoder()
y_train = le.fit_transform(train["G/L"])
y_test = le.transform(test["G/L"])

In [25]:
##Glimpse of y_train and y_test
print("\nThis is our transformed y_train as per current data:",y_train) 
print("\nThis is our transformed y_train as per current data:",y_test)


This is our transformed y_train as per current data: [ 0  1  2  3  4  5  6  7  7  7  7  7  7  7  7  8  9 10 11 12 13 14 15 16 17
 18 19 20 21 22 22 22 22 23 23 23 23 23 23 23 23 23 23 23 23 23]

This is our transformed y_train as per current data: [23 23 23 23 23  7  7  7  7 22 22  0]


# Machine Learning Models for Training/Testing
        SVC aka Support Vector Classifier
        Multinomial NB
        Random Forest Classifier
        Gaussian NB

In [42]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

### Initiating models 

In [43]:
print("\nInitializing models............")
print("\nSVC Initialized............")
svc= SVC(kernel='linear', probability=True, random_state=7)
print("\nMultinomialNB Initialized............")
nb = MultinomialNB()
print("\nRandomForestClassifier Initialized....")
rf = RandomForestClassifier(random_state=7)
print("\nGaussianNB Initialized............")
gNB = GaussianNB()
print(".....................................")
print("\nInitialization Succesful............")


Initializing models............

SVC Initialized............

MultinomialNB Initialized............

RandomForestClassifier Initialized....

GaussianNB Initialized............
.....................................

Initialization Succesful............


## Training the models

In [44]:
print("\n Started Training the models............")
print("\n SVC Trained............")
svc.fit(X_train, y_train)
print("\nMultinomialNB Trained............")
nb.fit(X_train, y_train)
print("\nRandomForestClassifier Trained....")
rf.fit(X_train, y_train)
print("\nGaussianNB Trained............")
gNB.fit(X_train.toarray(),y_train)
print(".....................................")
print("\nTraining is Succesful............")


 Started Training the models............

 SVC Trained............

MultinomialNB Trained............

RandomForestClassifier Trained....

GaussianNB Trained............
.....................................

Training is Succesful............


## Prediction with Test Data

In [45]:
print("\n Started Prediction with the models............")
print("\n")
print("\n SVC's Prediction over Test............")
svc.predict(X_test)


 Started Prediction with the models............



 SVC's Prediction over Test............


array([23, 23, 23, 23, 23,  7,  7,  7,  7, 22, 22,  0])

In [46]:
print("\nMultinomialNB's Prediction over Test............")
nb.predict(X_test)


MultinomialNB's Prediction over Test............


array([23, 23, 23, 23, 23, 23,  7, 23,  7, 22, 22, 23])

In [47]:
print("\nRandomForestClassifier Prediction over Test....")
rf.predict(X_test)


RandomForestClassifier Prediction over Test....


array([23, 23, 23, 23, 23,  7,  7,  7,  7, 22, 22,  0])

In [48]:
print("\nGaussianNB Prediction over Test............")
gNB.predict(X_test.toarray())


GaussianNB Prediction over Test............


array([23, 23, 23, 23, 23,  7,  7,  7,  7, 22, 22,  0])

In [49]:
print("\nTrue value of y_test is:............\n",y_test)


True value of y_test is:............
 [23 23 23 23 23  7  7  7  7 22 22  0]


#### Results can be easily depicted that models are performing pretty well and giving the matching IDs with good accuracy

##### Hence Prediction is succesful 

# Generating the Confidence Score for models
##### Confidence = Probability of the Prediction

In [61]:
print("\n Started Prediction with SVC model............")
print("\n")
svc_prob_pred = svc.predict_proba(X_test)
print("\nPrediction Scores")
svc.predict_proba(X_test)


 Started Prediction with SVC model............



Prediction Scores


array([[ 0.03133144,  0.03133249,  0.03133325,  0.03133374,  0.031334  ,
         0.03133404,  0.03133391,  0.00168831,  0.03133364,  0.03133327,
         0.03133285,  0.03133239,  0.03133195,  0.03133155,  0.03210381,
         0.031331  ,  0.03210344,  0.03210343,  0.03133093,  0.03133108,
         0.0313313 ,  0.03133157,  0.05154472,  0.28647191],
       [ 0.03223537,  0.03224313,  0.03225032,  0.03225684,  0.0322626 ,
         0.03226744,  0.03227122,  0.00151734,  0.03227398,  0.03227579,
         0.0322767 ,  0.03227681,  0.0322762 ,  0.03227499,  0.03190519,
         0.0322711 ,  0.03190065,  0.03189811,  0.03226368,  0.03226154,
         0.03226   ,  0.03225931,  0.04973579,  0.2722859 ],
       [ 0.03224487,  0.03225265,  0.03225986,  0.0322664 ,  0.03227216,
         0.03227702,  0.0322808 ,  0.00148221,  0.03228357,  0.03228538,
         0.0322863 ,  0.0322864 ,  0.03228579,  0.03228458,  0.03191438,
         0.03228069,  0.03190984,  0.0319073 ,  0.03227326,  0.03227112,
  

In [59]:
print("\n SVC's Top 3 Prediction ............")
svc_prob_pred.argsort()[-3:][::-1]


 SVC's Top 3 Prediction ............


array([[ 7, 23,  0, 17, 16, 14,  1,  2, 18, 19, 20,  3, 21, 15, 13,  4, 12,
         5, 11, 10,  6,  9,  8, 22],
       [ 7, 23, 17, 16, 14,  0,  1,  2, 19, 18, 20, 21,  3, 15, 13,  4, 12,
         5, 11, 10,  6,  9,  8, 22],
       [ 7, 23, 17, 16, 14,  0,  1,  2, 19, 18, 20, 21,  3, 15, 13,  4, 12,
         5, 11, 10,  6,  9,  8, 22]])

In [62]:
print("\n Started Prediction with MultinomialNB models............")
print("\n")
MNB_prob_pred = nb.predict_proba(X_test)
print("\nPrediction Scores")
nb.predict_proba(X_test)


 Started Prediction with MultinomialNB models............



Prediction Scores


array([[ 0.00636203,  0.00647547,  0.00647547,  0.00636203,  0.00636203,
         0.00636203,  0.00636203,  0.0714435 ,  0.00647547,  0.00625357,
         0.00636233,  0.00636233,  0.00636233,  0.00636203,  0.00610461,
         0.00636233,  0.00610461,  0.00610461,  0.00636203,  0.00636203,
         0.00636203,  0.00636203,  0.02513227,  0.77036082],
       [ 0.01118721,  0.01150506,  0.01150506,  0.01118721,  0.01118721,
         0.01118721,  0.01118721,  0.21778201,  0.01150506,  0.01088639,
         0.01118804,  0.01118804,  0.01118804,  0.01118721,  0.01047819,
         0.01118804,  0.01047819,  0.01047819,  0.01118721,  0.01118721,
         0.01118721,  0.01118721,  0.03090574,  0.51785183],
       [ 0.00336822,  0.00345444,  0.00345444,  0.00336822,  0.00336822,
         0.00336822,  0.00336822,  0.12963802,  0.00345444,  0.00328641,
         0.00336845,  0.00336845,  0.00336845,  0.00336822,  0.00317503,
         0.00336845,  0.00317503,  0.00317503,  0.00336822,  0.00336822,
  

In [64]:
print("\n MNB's Top 3 Prediction ............")
MNB_prob_pred.argsort()[-3:][::-1]


 MNB's Top 3 Prediction ............


array([[17, 16, 14,  9, 13, 20, 18, 19,  6,  5,  4,  3, 21, 10, 12, 15, 11,
         8,  2,  1,  0, 22,  7, 23],
       [17, 16, 14,  9,  0, 21, 20,  3,  4,  5,  6, 19, 18, 13, 15, 11, 10,
        12,  8,  2,  1,  7, 23, 22],
       [17, 16, 14,  9,  0, 21, 20,  3,  4,  5,  6, 19, 18, 13, 15, 11, 10,
        12,  8,  2,  1,  7, 23, 22]])

In [66]:
print("\n Started Prediction with RFM model............")
print("\n")
RFM_prob_pred = rf.predict_proba(X_test)
print("\nPrediction Scores")
rf.predict_proba(X_test)


 Started Prediction with RFM model............



Prediction Scores


array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  1. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
       

## Here you will be able to see the Greedy model performance

In [67]:
print("\n RFM's Top 3 Prediction ............")
RFM_prob_pred.argsort()[-3:][::-1]


 RFM's Top 3 Prediction ............


array([[11, 20, 19, 18, 17, 16, 14, 13, 12, 22, 23,  9,  7,  6,  5,  4,  3,
         2,  1, 10,  8, 15, 21,  0],
       [ 0, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,
         5,  4,  3,  2,  1, 23, 22],
       [ 0, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,
         5,  4,  3,  2,  1, 23, 22]])

In [69]:
print("\n Started Prediction with GNB model............")
print("\n")
GNB_prob_pred = gNB.predict_proba(X_test.toarray())
print("\nPrediction Scores")
gNB.predict_proba(X_test.toarray())


 Started Prediction with GNB model............



Prediction Scores


array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
      

In [70]:
print("\n GNB's Top 3 Prediction ............")
GNB_prob_pred.argsort()[-3:][::-1]


 GNB's Top 3 Prediction ............


array([[11, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 23, 10,  9,  8,  7,  6,
         5,  4,  3,  2,  1, 22,  0],
       [ 0, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,
         5,  4,  3,  2,  1, 23, 22],
       [ 0, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,
         5,  4,  3,  2,  1, 23, 22]])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, GNB_prob_pred)